In [1]:
import pandas as pd
import numpy as np

In [2]:
#val_results = pd.read_csv('../doc/models/model_02/model_02.csv')
#val_results = pd.read_csv('../../doc/models/model_02r/model_02r.csv')
val_results = pd.read_csv('../../doc/models/model_02s/model_02s.csv')
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score
0,sub-100247,ses-V02,1,T2w,sub-100247_ses-V02_run-1_T2w.nii.gz,3.0,NaN
1,sub-100524,ses-V02,1,T2w,sub-100524_ses-V02_run-1_T2w.nii.gz,1.0,NaN
2,sub-100586,ses-V02,1,T2w,sub-100586_ses-V02_run-1_T2w.nii.gz,0.0,NaN
3,sub-100642,ses-V02,1,T2w,sub-100642_ses-V02_run-1_T2w.nii.gz,0.0,NaN
4,sub-100642,ses-V02,1,T1w,sub-100642_ses-V02_run-1_T1w.nii.gz,0.5,NaN


In [3]:
# find scans with highest prediction error
val_results['abs_error'] = abs(val_results['QU_motion'] - val_results['predicted_qu_motion_score'])
val_results = val_results.sort_values(by='abs_error', ascending=False)
val_results.head(15)

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
146,sub-118437,ses-V03,2,T2w,sub-118437_ses-V03_run-2_T2w.nii.gz,3.0,0.079878,2.920122
344,sub-142412,ses-V02,3,T1w,sub-142412_ses-V02_run-3_T1w.nii.gz,3.0,0.236227,2.763773
671,sub-181584,ses-V04,1,T2w,sub-181584_ses-V04_run-1_T2w.nii.gz,3.0,0.388812,2.611188
580,sub-167328,ses-V04,1,T1w,sub-167328_ses-V04_run-1_T1w.nii.gz,1.0,-1.588155,2.588155
224,sub-129599,ses-V02,1,T2w,sub-129599_ses-V02_run-1_T2w.nii.gz,3.0,0.545866,2.454134
753,sub-186863,ses-V03,1,T2w,sub-186863_ses-V03_run-1_T2w.nii.gz,3.0,0.724301,2.275699
342,sub-142412,ses-V02,1,T1w,sub-142412_ses-V02_run-1_T1w.nii.gz,3.0,0.836286,2.163714
471,sub-154414,ses-V03,1,T2w,sub-154414_ses-V03_run-1_T2w.nii.gz,3.0,0.993847,2.006153
343,sub-142412,ses-V02,2,T1w,sub-142412_ses-V02_run-2_T1w.nii.gz,3.0,1.000091,1.999909
544,sub-161713,ses-V04,1,T2w,sub-161713_ses-V04_run-1_T2w.nii.gz,3.0,1.017120,1.982880


In [4]:
# save first 15 rows to csv
val_results.head(15).to_csv('../../data/validation_errors/model_02s/validation_set_top15_errors.csv', index=False)

In [5]:
# remove the 5 highest error scans
val_results = val_results.iloc[5:]
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
753,sub-186863,ses-V03,1,T2w,sub-186863_ses-V03_run-1_T2w.nii.gz,3.0,0.724301,2.275699
342,sub-142412,ses-V02,1,T1w,sub-142412_ses-V02_run-1_T1w.nii.gz,3.0,0.836286,2.163714
471,sub-154414,ses-V03,1,T2w,sub-154414_ses-V03_run-1_T2w.nii.gz,3.0,0.993847,2.006153
343,sub-142412,ses-V02,2,T1w,sub-142412_ses-V02_run-2_T1w.nii.gz,3.0,1.000091,1.999909
544,sub-161713,ses-V04,1,T2w,sub-161713_ses-V04_run-1_T2w.nii.gz,3.0,1.017120,1.982880


In [6]:
# count NaN values in abs_error column
val_results['abs_error'].isna().sum()

np.int64(812)

In [7]:
# drop rows with NaN values in abs_error column
val_results = val_results.dropna(subset=['abs_error'])
val_results['abs_error'].isna().sum()

np.int64(0)

In [8]:
def create_correlation_coefficient(actual_vals, predicted_vals):
    """Calculate Pearson correlation coefficient between actual and predicted values.

    Args:
        actual_vals (list): Actual QU_motion scores
        predicted_vals (list): Predicted QU_motion scores

    Returns:
        float: Pearson correlation coefficient (r value)
    """
    x = np.array(actual_vals)
    y = np.array(predicted_vals)

    correlation_matrix = np.corrcoef(x, y)
    correlation_coefficient = correlation_matrix[0, 1]

    return correlation_coefficient

In [9]:
# recalculate metrics without the 5 highest error scans
corr = create_correlation_coefficient(val_results['QU_motion'], val_results['predicted_qu_motion_score'])
corr

np.float64(0.49226961039374734)